In [ ]:
import sys
sys.path.append('../src')

In [ ]:
import time, os, sys
import tqdm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import gta
import importlib
importlib.reload(gta)

## Record gameplay.
Press `escape` to stop the recording, after some keyboard inputs.

In [ ]:
win = gta.GTA()
win.SetAsForegroundWindow()
time.sleep(4)
win.startRecordingKeyboard()
win.grabImagesUntilInterrupted()

In [ ]:
len(win.imgs)

In [ ]:
len(win.joystickRecorder.events)

In [ ]:
self = win

In [ ]:
def save(self, fpath=None, compressed=False):
    start = time.time()
    if fpath is None:
        fpath = os.path.join(
            os.path.expanduser('~'),
            'data',
            'gta%s.npz' % start
            )

    
    gta.mkdir_p(os.path.dirname(fpath))

    tosave = dict(
        imgs=self.imgs,
        imgTimes=self.imgTimes,
        numericRecords=[],
        **{k: np.stack(v) for (k,v) in self.joystickRecorder.getEvents().items()},
    )
    if len(self.keyRecord) > 0:
        toSave['numericRecords'] = np.vstack(self.keyRecord)
    
    print('Saving to %s ... ' % fpath, end=''); sys.stdout.flush()
    if compressed:
        np.savez_compressed(fpath, **tosave)
    else:
        np.savez(fpath, **tosave)
    print('done (elapsed %.0f s).' % (time.time() - start,))

In [ ]:
print('Images are %.2f GB' % (win.imgs.size * win.imgs.dtype.itemsize / 1024 / 1024 / 1024,))

In [ ]:
save(win, compressed=False)

Takes about 4 times as long if compression is indicated.

In [ ]:
save(win, compressed=True)

In [ ]:
win.save()

## Load and use data.

In [ ]:
states = win.states
nimgs = 4*6
startIndex = 0
imgs = win.imgs[startIndex:startIndex+nimgs]
imgTimes = win.imgTimes[startIndex:startIndex+nimgs]
statest = states[startIndex:startIndex+nimgs]

In [ ]:
keyNames = list(win.keyNames); keyNames.append('other')
ncols = 4
nrows = int(len(imgs) / ncols)
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 3*nrows))
iimg = 0
for i in range(nrows):
    for j in range(ncols):
        ax = axes[i][j]
        ax.imshow(imgs[iimg])
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(
            '$t=%.3f$ seconds' % (imgTimes[iimg] - imgTimes[0],)
            +'\nkeys: "%s"=(%s)' % (
                ''.join([name for (name, keydown) in zip(keyNames, statest[iimg]) if keydown]),
                ' '.join(['%d' % s for s in statest[iimg]]),
            )
        )
        iimg += 1
fig.tight_layout()
fig.subplots_adjust(wspace=0, hspace=.0)
fig.savefig('tensorflowGta.png')